In [1]:
import os

import numpy as np
import pandas as pd

from models.decisionTree import DecisionTree
from models.PRISM_Algorithm import PRISM_Algorithm

In [23]:
data_path = './data'
data_file = 'fase1_20112019.xlsx'
sheets = ['ECONOMÍA', 'INMIGRACIÓN']
model = 0  # 0: Decision Tree --- 1: PRISM Rules ---

useful_columns = ['ID', 'Column1', 'COM. CONSTRUCTIU', 'COM. TÒXIC', 'GRAU TOXICITAT', 'Sarcasme/Ironia',
                  'Burla/ridiculització', 'Insults', 'Argumentació/Diàleg', 'Llenguatge negatiu/tòxic']

columns = ['TextData', 'Constructive', 'Toxic', 'ToxicityDegree', 'Sarcasm_Irony',
           'Mockery_Ridicule', 'Insults', 'Argument_Discussion', 'NegativeToxicLanguage']

data = pd.DataFrame()
for sheet in sheets:
    # Read data
    new_data = pd.read_excel(os.path.join(data_path, data_file), sheet_name=sheet, index_col='ID',
                             usecols=useful_columns)
    # Rename the columns
    new_data.columns = columns

    data = pd.concat([data, new_data], ignore_index=True)

# PREPROCESS THE DATA
data['ToxicityDegree'] = data[['ToxicityDegree']].fillna(1)
data = data.drop(['Toxic'], axis=1)
columns.pop(columns.index('Toxic'))

data.dropna(inplace=True)

data.Insults = data.Insults.str.upper()
data['Sarcasm_Irony'].replace({'SI': 'SÍ'}, inplace=True)

y_label = 'ToxicityDegree'

# TEST with MANUAL data
manual_feats = columns[1:]
manual_data = data[manual_feats]

In [12]:
pd.options.display.max_colwidth = 1000
data.query('Insults == "SÍ" and ToxicityDegree == 1')#[manual_data.Insults == 'SÍ']

,TextData,Constructive,ToxicityDegree,Sarcasm/Irony,Mockery/Ridicule,Insults,Argument/Discussion,NegativeToxicLanguage
218,"Un día en la cola del super dos yonkis tenían la siguiente conversación: - Antes las cosas eran distintas, una empresa quería librarse sin pagar de materiales que sobraban y dejaban la verja sin cerrar, nosotros entrábamos, cogíamos eso sin tocar nada más y nos íbamos, todos ganábamos. Luego llegaron los rumanos y cuando veían algo mal cerrado se llevaban hasta la tele. - Si tío, hasta la tele, qué poca vergüenza! - Y claro, ahora han tenido que poner vigilancia, nosotros no podemos llevar metal y la empresa tiene que molestarse en librarse de esas cosas. Todo por culpa de los rumanos -Sí tío, putos rumanos, no tienen moral.",SÍ,1.0,NO,NO,SÍ,NO,SÍ


In [13]:
data.query('Constructive == "SÍ" and ToxicityDegree != 1')

,TextData,Constructive,ToxicityDegree,Sarcasm/Irony,Mockery/Ridicule,Insults,Argument/Discussion,NegativeToxicLanguage
1,"Se ve que ya han refundado el capitalismo sobre bases éticas, que decía Sarkozy allá por 2008. El mismo que lideró la invasión que llevó la democracia a Libia",SÍ,3.0,SÍ,NO,NO,NO,NO
22,"Ya nos descojonamos con sus sistema sanitario de hecho medio planeta se ríe, pudiendo ser el mejor sistema sanitario es el mas estúpido jamas creado.",SÍ,3.0,NO,SÍ,NO,NO,SÍ
24,Siempre les puedes comentar que paso con enron lemans y demás mierda que usaban para sus planes de pensiones se amortizasen.,SÍ,3.0,NO,NO,NO,NO,SÍ
28,"que pregunten también en Chile por los planes privados de pensiones, creo que están muy contentos",SÍ,3.0,SÍ,SÍ,NO,NO,NO
29,"Aquí tenemos patriotas de pacotilla de esos que cuentas, y otros que se dicen patriotas también, y hablan de mejorar las pensiones pero se alinean, cuando no lideran, movimientos a favor de dar más dineros a las regiones más ricas.",SÍ,2.0,NO,NO,NO,SÍ,NO
32,Menuda chorrada churrimerinista. Las pensiones se asignan por individuo y no por territorio. ¿Acaso tú verías normal que los territorios con más población recibieran el mismo dinero para pensiones que los que tienen menos población? ¿Sus pensionistas son ricos porque tú ha decidido que es un territorio rico sólo porque conjuntamente cotizan más?,SÍ,2.0,NO,NO,NO,SÍ,SÍ
34,"Tienes razón, las pensiones son a título individual y así deben ser consideradas. Sin embargo, a menudo el punto de vista de las regiones se usa para enmierdar el debate, sobre todo cuando se sueltan datos sobre número de cotizantes frente a pensionistas, acusando sibilinamente a las zonas más pobres de vivir a cuenta de las más ricas. Sin embargo, se obvia que muchos de esos pensionistas actuales cotizaron toda su vida en esas regiones ricas (contando como en su momento activos),",SÍ,2.0,NO,NO,NO,SÍ,NO
35,"Te inventas un ”enmierdamiento” que sólo estás propugnando tú. Yo jamás he visto A NADIE quejarse de que unos territorios gasten más que otros en pensiones (si habláramos de subsidios o subvenciones sería otro cantar, pero en pensiones no). De hecho las PENSIONES DE JUBILACIÓN no son una competencia cedida a las CCAA y su presencia en los presupuestos autonómicos es cero, lo cual hace que toda tu argumentación sea una basura inventada por ti.",SÍ,2.0,NO,NO,NO,SÍ,SÍ
44,"Menuda majadería. No hay más patriota que aquel cuya patria son sus padres, su mujer y sus hijos. Esa es la verdadera y única patria de cada uno.",SÍ,2.0,NO,NO,NO,NO,SÍ
45,"Y en cuanto a los planes de pensiones, el sistema piramidal actual es una (otra más) estafa que nos dejó como manzana envenenada uno de los mayores trileros de la España moderna, el sociolisto Felipe. Yo no quiero pagar las pensiones de nadie. Quiero que el estado deje de robarme para poder pagarme la mía",SÍ,2.0,NO,NO,NO,SÍ,SÍ


In [17]:
from sklearn.metrics import normalized_mutual_info_score

for ix1 in range(len(manual_feats)):
    col1 = manual_feats[ix1]
    for ix2 in range(ix1+1, len(manual_feats)):
        col2 = manual_feats[ix2]
        print('AMI for', col1, 'and', col2, 'is:', normalized_mutual_info_score(manual_data[col1], manual_data[col2],
                                                                               average_method='arithmetic'))

AMI for Constructive and ToxicityDegree is: 0.06096767232664145
AMI for Constructive and Sarcasm/Irony is: 0.04847843869499998
AMI for Constructive and Mockery/Ridicule is: 0.10455181973493183
AMI for Constructive and Insults is: 0.03645127863459253
AMI for Constructive and Argument/Discussion is: 0.17954623187131935
AMI for Constructive and NegativeToxicLanguage is: 0.0819250319184821
AMI for ToxicityDegree and Sarcasm/Irony is: 0.026181298393234423
AMI for ToxicityDegree and Mockery/Ridicule is: 0.04503727769763158
AMI for ToxicityDegree and Insults is: 0.20046230592874914
AMI for ToxicityDegree and Argument/Discussion is: 0.025199330601388206
AMI for ToxicityDegree and NegativeToxicLanguage is: 0.22841848067519774
AMI for Sarcasm/Irony and Mockery/Ridicule is: 0.18585512554245673
AMI for Sarcasm/Irony and Insults is: 0.0043463234973132245
AMI for Sarcasm/Irony and Argument/Discussion is: 0.05552133986062071
AMI for Sarcasm/Irony and NegativeToxicLanguage is: 0.0029226951898740855
AM

In [19]:
from sklearn.metrics import adjusted_rand_score

for ix1 in range(len(manual_feats)):
    col1 = manual_feats[ix1]
    for ix2 in range(ix1+1, len(manual_feats)):
        col2 = manual_feats[ix2]
        print('ARI for', col1, 'and', col2, 'is:', adjusted_rand_score(manual_data[col1], manual_data[col2]))

ARI for Constructive and ToxicityDegree is: 0.1390579095356588
ARI for Constructive and Sarcasm/Irony is: 0.10736241001314922
ARI for Constructive and Mockery/Ridicule is: 0.168010557168838
ARI for Constructive and Insults is: 0.06109935439134069
ARI for Constructive and Argument/Discussion is: 0.14511424217798766
ARI for Constructive and NegativeToxicLanguage is: 0.16966370855164906
ARI for ToxicityDegree and Sarcasm/Irony is: 0.10216746528459064
ARI for ToxicityDegree and Mockery/Ridicule is: 0.14145690797368285
ARI for ToxicityDegree and Insults is: 0.17629240637646065
ARI for ToxicityDegree and Argument/Discussion is: -0.00207044664197594
ARI for ToxicityDegree and NegativeToxicLanguage is: 0.4119281531781824
ARI for Sarcasm/Irony and Mockery/Ridicule is: 0.37864242757131844
ARI for Sarcasm/Irony and Insults is: -0.03202671116121232
ARI for Sarcasm/Irony and Argument/Discussion is: 4.839210728158963e-05
ARI for Sarcasm/Irony and NegativeToxicLanguage is: -0.02588813960217248
ARI fo

In [26]:
data.query('Insults == "SÍ" and NegativeToxicLanguage == "SÍ"')

,TextData,Constructive,ToxicityDegree,Sarcasm_Irony,Mockery_Ridicule,Insults,Argument_Discussion,NegativeToxicLanguage
23,"Si, una puta mierda el sistema sanitario americano, pero eso no invalida lo que dije.",NO,3.0,NO,NO,SÍ,NO,SÍ
46,"Tu si que eres un lerdo, neoliberal de palo. Los anglos te desprecian y te escupirian a la cara si los tuvieras de frente. Para ellos eres un zingaro. No hay ideologia más criminal que el neoliberalismo",NO,4.0,NO,NO,SÍ,NO,SÍ
49,Jajajaja. Me rio yo del chupapollas de Rallo y sus seguidores,NO,4.0,NO,NO,SÍ,NO,SÍ
67,"Date cuenta que dejan caer que estan pensando hacerlo obligatorio por ley, los muy hijos de puta. No se que decir, de verdad, esto seria un atraco a mano armada y consentido/auspiciado por el Estado. Y cuando te jubiles iran y quebraran y que te den por saco; es posible que muchos no solo salgan perdiendo con la inversion en el dichoso plan, sino que no vean ni un euro de su dinero. Ladrones.",SÍ,3.0,NO,NO,SÍ,SÍ,SÍ
71,"Y aún habrá desgraciados que defiendan esto. Antes el Estado, que se supone somos todos, respondía ante la ciudadanía. Ahora esa ciudadanía dependerá de un consejo de administración. Muy alentador.",SÍ,3.0,NO,NO,SÍ,SÍ,SÍ
106,A todo trabajador de derechas.......JODEROS ......me alegro aun nos pasa poco.... animo !!!!!,NO,4.0,SÍ,SÍ,SÍ,NO,SÍ
113,"Vaya panda de catetos que conforma meneame. En UK es obligatorio tener un plan de pensiones privado, funciona perfectamente y es la manera mas justa de obtener la jubilación que te mereces conforme a lo que has ganado mientras trabajaste (y has decidido aportar). Las contribuciones son a partir de tu sueldo bruto y cuando llega la hora de rescatarlo (a partir de los 55 ya puedes) tienes flexibidad total. Puedes sacar un 25% de golpe libre de impuestos, ir sacando el dinero a medida que lo necesites o comprar una renta anual de por vida. Mucho mejor esto a que el gobierno me diga lo maximo que puede ser mi pension",SÍ,3.0,NO,NO,SÍ,SÍ,SÍ
155,"Al menos en Francia tienen el Frente Nacional, ¿ pero aquí ?. Nos estamos convirtiendo en la reserva espiritual del giliprogresismo mas autodestructivo. Autodestructivo para la clase obrera.",NO,4.0,NO,SÍ,SÍ,NO,SÍ
186,Deja de buscar excusas en el sistema y de ser un puto vagx,NO,4.0,NO,NO,SÍ,NO,SÍ
218,"Un día en la cola del super dos yonkis tenían la siguiente conversación: - Antes las cosas eran distintas, una empresa quería librarse sin pagar de materiales que sobraban y dejaban la verja sin cerrar, nosotros entrábamos, cogíamos eso sin tocar nada más y nos íbamos, todos ganábamos. Luego llegaron los rumanos y cuando veían algo mal cerrado se llevaban hasta la tele. - Si tío, hasta la tele, qué poca vergüenza! - Y claro, ahora han tenido que poner vigilancia, nosotros no podemos llevar metal y la empresa tiene que molestarse en librarse de esas cosas. Todo por culpa de los rumanos -Sí tío, putos rumanos, no tienen moral.",SÍ,1.0,NO,NO,SÍ,NO,SÍ


La fila 354 tiene un grado de toxicidad bastante bajo para lo que es